---

**Load essential libraries**

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

import tensorflow as tf

---

**Check TensorFlow version**

---

In [ ]:
tf.__version__

'2.15.0'

---

Load MNIST Data

---

In [ ]:
## Load MNIST data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

num_labels = len(np.unique(y_train))
num_features = X_train.shape[1]
num_samples = X_train.shape[0]

# One-hot encode class labels
Y_train = tf.keras.utils.to_categorical(y_train)
Y_test = tf.keras.utils.to_categorical(y_test)

# Normalize the samples (images)
xmax = np.amax(X_train)
xmin = np.amin(X_train)
X_train = (X_train - xmin) / (xmax - xmin) # all train features turn into a number between 0 and 1
X_test = (X_test - xmin)/(xmax - xmin)

print('MNIST set')
print('---------------------')
print('Number of training samples = %d'%(num_samples))
print('Number of features = %d'%(num_features))
print('Number of output labels = %d'%(num_labels))

11490434/11490434 [==============================] - 1s 0us/step
MNIST set
---------------------
Number of training samples = 60000
Number of features = 784
Number of output labels = 10


---

We will now look at 3 different ways to build custom models using TensorFlow 2:

1. model subclassing ([Making new layers and models via subclassing](https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing))
2. sequential API
3. functional API

---

---

**Approach-1**: here we build the model by subclassing the Keras $\texttt{Model}$ class followed by definition of of layers in $\texttt{__init__}$ and implementation of the model's forward pass in $\texttt{call()}$.

---

In [ ]:
## Define 1-layer (softmax) neural network architecture
# Define model
class Softmax_Model(tf.keras.models.Model):
    def __init__(self):
        super(Softmax_Model, self).__init__()
        initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)
        self.dense1 = tf.keras.layers.Dense(num_labels, dtype = 'float64',\
                                 bias_initializer = initializer,\
                                 activation = tf.keras.activations.softmax)

    # Forward pass for the model
    def call(self, inputs):
        a = self.dense1(inputs)
        return a

---

Build model

---

In [ ]:
## Build model
model = Softmax_Model()
batch_size = 100 # batch size
model.build((batch_size, num_features))

---

Compile and train the model on the training batches and test on the test set in one shot

---

In [ ]:
## Compile model
opt = tf.keras.optimizers.Adam(learning_rate = 1e-03) # optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # loss function
model.compile(optimizer = opt, loss = loss_fn, metrics = ['acc'])

# Train model and simultabeously test on the test set
model.fit(X_train, Y_train, batch_size = 100,\
          epochs = 10,\
          validation_data = (X_test, Y_test))

Epoch 1/10
600/600 [==============================] - 4s 5ms/step - loss: 0.6472 - acc: 0.8357 - val_loss: 0.3669 - val_acc: 0.9058
Epoch 2/10
600/600 [==============================] - 2s 3ms/step - loss: 0.3500 - acc: 0.9051 - val_loss: 0.3083 - val_acc: 0.9158
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 0.3117 - acc: 0.9143 - val_loss: 0.2921 - val_acc: 0.9185
Epoch 4/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2938 - acc: 0.9184 - val_loss: 0.2812 - val_acc: 0.9220
Epoch 5/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2832 - acc: 0.9217 - val_loss: 0.2766 - val_acc: 0.9217
Epoch 6/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2758 - acc: 0.9235 - val_loss: 0.2712 - val_acc: 0.9245
Epoch 7/10
600/600 [==============================] - 3s 4ms/step - loss: 0.2703 - acc: 0.9250 - val_loss: 0.2679 - val_acc: 0.9238
Epoch 8/10
600/600 [==============================] - 2s 3ms/step - loss: 0.

---

Instead of doing the above, we can explicitly write down the optimization step using $\texttt{GradientTape()}$ and train the model

---

In [ ]:
## Create source dataset from input data (this is helpful for ppipelining later)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
batch_size = 100 # batch size
# Create training batches
train_dataset = train_dataset.shuffle(buffer_size = 1024).batch(batch_size)

In [ ]:
# Create softmax model
model = Softmax_Model()

opt = tf.keras.optimizers.Adam(learning_rate = 1e-03) # optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # loss function

# Varible to store training loss per epoch
loss_train_epoch = tf.keras.metrics.Mean()

# Iterate over epochs
nepochs = 10
for epoch in range(nepochs):
  # Iterate over the batches of the dataset.
  for step, train_batch in enumerate(train_dataset):
    with tf.GradientTape() as g:
      # Compute loss
      yhat = model(train_batch[0])
      loss = loss_fn(train_batch[1], yhat)

    # Calculate gradients
    grad = g.gradient(loss, model.trainable_weights)

    # Update model
    opt.apply_gradients(zip(grad, model.trainable_weights))

    # Append training loss
    loss_train_epoch(loss)
  print('Epoch %d: train loss = %f'%(epoch+1, loss_train_epoch.result()))

Epoch 1: train loss = 0.639664
Epoch 2: train loss = 0.494852
Epoch 3: train loss = 0.434000
Epoch 4: train loss = 0.399185
Epoch 5: train loss = 0.376163
Epoch 6: train loss = 0.359586
Epoch 7: train loss = 0.346981
Epoch 8: train loss = 0.336987
Epoch 9: train loss = 0.328823
Epoch 10: train loss = 0.322007


In [ ]:
# Compile model so it can be evaluated for test set
model.compile(optimizer = opt, loss = loss_fn, metrics = ['acc'])
print('\nAccuracy:', model.evaluate(X_test, Y_test, verbose=0)[1])


Accuracy: 0.9259999990463257


---

**Approach-2**: here we build the model using the sequential API of TensorFlow Keras. Try this.

---

In [ ]:
# prompt: here we build the model using the sequential API of TensorFlow Keras. Try this.

# Define 1-layer (softmax) neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(num_labels, activation='softmax', dtype = 'float64', input_shape=(num_features,))
])

# Compile model
opt = tf.keras.optimizers.Adam(learning_rate = 1e-03) # optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # loss function
model.compile(optimizer = opt, loss = loss_fn, metrics = ['acc'])

# Train model and simultabeously test on the test set in one shot
model.fit(X_train, Y_train, batch_size = 100,\
          epochs = 10,\
          validation_data = (X_test, Y_test))


Epoch 1/10
600/600 [==============================] - 4s 5ms/step - loss: 0.6330 - acc: 0.8381 - val_loss: 0.3616 - val_acc: 0.9062
Epoch 2/10
600/600 [==============================] - 2s 3ms/step - loss: 0.3458 - acc: 0.9057 - val_loss: 0.3083 - val_acc: 0.9135
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 0.3090 - acc: 0.9147 - val_loss: 0.2902 - val_acc: 0.9187
Epoch 4/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2921 - acc: 0.9188 - val_loss: 0.2803 - val_acc: 0.9215
Epoch 5/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2816 - acc: 0.9214 - val_loss: 0.2743 - val_acc: 0.9239
Epoch 6/10
600/600 [==============================] - 2s 4ms/step - loss: 0.2744 - acc: 0.9234 - val_loss: 0.2737 - val_acc: 0.9230
Epoch 7/10
600/600 [==============================] - 3s 4ms/step - loss: 0.2690 - acc: 0.9248 - val_loss: 0.2725 - val_acc: 0.9233
Epoch 8/10
600/600 [==============================] - 2s 3ms/step - loss: 0.

---

**Approach-3**: here we build the model using the functional API of TensorFlow Keras. Try this.

---

In [ ]:
# prompt: here we build the model using the functional API of TensorFlow Keras. Try this.

# Define input layer
input = tf.keras.Input(shape=(num_features,))
# Define Dense layer
dense = tf.keras.layers.Dense(num_labels, activation='softmax', dtype = 'float64')
# Connect input layer to dense layer
yhat = dense(input)

# Define model
model = tf.keras.Model(input, yhat)

# Compile model
opt = tf.keras.optimizers.Adam(learning_rate = 1e-03) # optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # loss function
model.compile(optimizer = opt, loss = loss_fn, metrics = ['acc'])

# Train model and simultabeously test on the test set in one shot
model.fit(X_train, Y_train, batch_size = 100,\
          epochs = 10,\
          validation_data = (X_test, Y_test))


Epoch 1/10
600/600 [==============================] - 3s 4ms/step - loss: 0.6296 - acc: 0.8427 - val_loss: 0.3654 - val_acc: 0.9038
Epoch 2/10
600/600 [==============================] - 2s 4ms/step - loss: 0.3473 - acc: 0.9055 - val_loss: 0.3105 - val_acc: 0.9152
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 0.3098 - acc: 0.9144 - val_loss: 0.2896 - val_acc: 0.9186
Epoch 4/10
600/600 [==============================] - 3s 4ms/step - loss: 0.2922 - acc: 0.9180 - val_loss: 0.2795 - val_acc: 0.9239
Epoch 5/10
600/600 [==============================] - 2s 4ms/step - loss: 0.2821 - acc: 0.9209 - val_loss: 0.2748 - val_acc: 0.9237
Epoch 6/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2750 - acc: 0.9227 - val_loss: 0.2706 - val_acc: 0.9247
Epoch 7/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2699 - acc: 0.9246 - val_loss: 0.2681 - val_acc: 0.9257
Epoch 8/10
600/600 [==============================] - 2s 3ms/step - loss: 0.